# Home Credit

## 배경이해

Home Credit 은 1997년 체코에서 설립된 국제적인 비은행 금융 기관입니다. 14개국에서 운영되고 있으며, 신용기록이 적거나 전혀 없는 사람들을 대상으로한 대출에 중점을 두고있습니다. 보통 신용기록이 없는 사람들은 안정적인 금융권에서 대출을 받는데 어려움이 많아 고리의 대출을 이용하게 됩니다. Home Credit에서는 다양한 대체 데이터(통신 혹은 거래정보 등)와 다양한 통계 및 머신러닝 기법을 활용해 고객의 상환 능력을 예측하여 이런 사람들에게도 긍정적인 대출 경험을 제공하려 노력하고 있습니다.<br>

Home Credit은 본 대회를 통해 상환할 수 있는 고객의 대출 신청을 거절하지 않고, 고객이 충분히 상환할 수 있는 적절한 원금, 만기 및 상환일정을 예측하는 등 데이터의 잠재력을 끌어낼 수 있기를 바란다고 합니다.<br>

* application_{train|test}.csv : home credit의 각 대출 application 정보가 있다. 모든 대출에는 자체 행이 있고 SK_ID_CURR로 식별된다. Train 데이터는 대출이 상환되었거나 0, 상환되지 않는 (1)이 제공된다.
<br>
* previous_application.csv : application data에 대출이 있는 고객의 previous application for loan, 여러 개의 이전 대출을 가질 수 있다.
<br>
* bureau.csv : 다른 금융 기관의 고객의 이전 신용 데이터, 각 이전 신용은 자체적인 row가 있지만 applicaion data에서 하나의 대출은 여러개의 previous credit을 가질 수 있다.
<br>
* bureau_balance.csv : 위 bureau.csv의 월별 데이터 이다. 각 행은 한 달을 말하고 매월 한 행씩 여러행을 가진다.
<br>
* POS_CASH_balance.csv : 고객의 point of sale, 현금 대출에 대한 월별 데이터, 각 행은 한 달을 뜻하고 하나의 대출은 여러 행을 가질 수 있다.
<br>
* credit_card_balance.csv : 신용카드 위협 데이터, 각 행은 신용카드의 잔액 한 달 데이터를 뜻한다.
<br>
* installments_payments.csv : 이전 대출에서 payment 히스토리.
<br>
* HomeCredit_columns_description.csv : 이 파일은 다양한 데이터 파일의 열에 대한 설명을 포함하고 있다.

In [2]:
# File system manangement
import os
import numpy as np
import pandas as pd
import seaborn as sns
import missingno as msno
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

# Data Load

In [8]:
print(os.listdir("home_credit_data/"))

['application_test.csv', 'HomeCredit_columns_description.csv', 'POS_CASH_balance.csv', 'credit_card_balance.csv', 'installments_payments.csv', 'application_train.csv', 'bureau.csv', 'previous_application.csv', 'bureau_balance.csv', 'sample_submission.csv']


In [10]:
train = pd.read_csv("home_credit_data/application_train.csv")
test = pd.read_csv("home_credit_data/application_test.csv")
print(train.shape)
print(test.shape)

(307511, 122)
(48744, 121)


In [11]:
train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
